<a href="https://colab.research.google.com/github/kuchcakova/NER_XAI/blob/main/XAI_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !python --version
# !sudo apt-get update -y
# !sudo apt-get install -y python3.8 python3.8-distutils python3-pip
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
# !sudo update-alternatives --config python3
# !python3 --version

##python decreasing version to 3.8


In [ ]:
#!pip install scikit-learn==1.4

#!pip uninstall scikit-learn -y

!pip3 install scikit-learn==1.2.0

import sklearn
print(sklearn.__version__)

1.2.0


In [ ]:
!pip3 install pyngrok
!pip3 install gdown
!pip3 install PyMuPDF
!pip3 install eli5
!pip3 install transformers_interpret

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
##network tunel delete

# from pyngrok import ngrok
# active_tunnels = ngrok.get_tunnels()

# for tunnel in active_tunnels:
#     ngrok.disconnect(tunnel.public_url)

# ngrok.kill()

##for delete downlaoded local model

# def remove_google_model():
#     extract_path_model = './model_files'
#     extract_path_model_zip = './downloaded_model.zip'

#     if os.path.exists(extract_path_model):
#         try:
#             shutil.rmtree(extract_path_model)
#             print("Directory and contents deleted successfully!")
#         except Exception as e:
#             print(f"Failed to delete directory due to: {e}")
#     else:
#         print("Directory does not exist.")

#     if os.path.exists(extract_path_model_zip):
#         try:
#             os.remove(extract_path_model_zip)
#             print("Directory and contents deleted successfully!")
#         except Exception as e:
#             print(f"Failed to delete directory due to: {e}")
#     else:
#         print("Directory does not exist.")

#     return redirect(url_for('index'))


In [ ]:
def align_masked_tokens(original_tokens, masked_tokens):
    """
    Aligns the masked_tokens list by ensuring that spans replaced with 'UN' or 'ĠUN'
    match the length of the corresponding spans in the original_tokens list.

    This modifies a copy of the masked_tokens list directly.
    """

    print("Original tokens: ",original_tokens)
    print("Masked tokens: ",masked_tokens)

    aligned_tokens = []
    i, j = 0, 0  # Pointers for original and masked tokens

    while i < len(original_tokens) and j < len(masked_tokens):
        if masked_tokens[j] in ['UN', 'ĠUN']:  # Found a masked token
            # Add the first UN or ĠUN
            aligned_tokens.append(masked_tokens[j])
            j_before = j

            # Find token after the masked span
            while j < len(masked_tokens) and masked_tokens[j] in ['UN', 'ĠUN']:
                j += 1

            # If we hit the end of the masked tokens list, stop to avoid an infinite loop
            if j >= len(masked_tokens):
                break

            # The token after the UN span
            token_after = masked_tokens[j]

            #move i to next token
            if i<len(original_tokens):
                i+=1

            # Move i to align original tokens until the token_after is matched
            number_of_inserted_tokens = 0
            while i < len(original_tokens) and original_tokens[i] != token_after:
                number_of_inserted_tokens += 1
                i += 1

                # Safety check: Break if token_after doesn't exist in original_tokens
                if number_of_inserted_tokens > len(original_tokens):
                    raise ValueError("Alignment failed: token_after not found in original_tokens")

            # Insert 'UN' tokens for the unmatched span
            aligned_tokens.extend(['UN'] * number_of_inserted_tokens)


        else:
            # If no masking, just copy the token
            aligned_tokens.append(masked_tokens[j])
            i += 1
            j += 1

    # Append any remaining masked tokens (if present)
    while j < len(masked_tokens):
        aligned_tokens.append(masked_tokens[j])
        j += 1

    if len(aligned_tokens) != len(original_tokens):
        print("Aligned tokens: ",aligned_tokens)
        print("Original tokens ",original_tokens)
        raise ValueError("Alignment failed: lengths of aligned tokens and original tokens do not match")


    return aligned_tokens


In [ ]:
import scipy
import torch
import numpy as np
from tqdm.auto import tqdm
from typing import List
import pandas as pd

import gdown
import zipfile
import shutil
import os
import re

from google.colab import files
import time
from flask import Flask, render_template, request, redirect, url_for, flash, send_file, redirect, jsonify, Response, session #Markup
from pyngrok import ngrok

from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

def monkeypath_itemfreq(sampler_indices):
    return zip(*np.unique(sampler_indices, return_counts=True))

scipy.stats.itemfreq=monkeypath_itemfreq

from eli5.lime import TextExplainer
from eli5.lime.samplers import MaskingTextSampler
from eli5.lime.textutils import TokenizedText

import fitz
from transformers_interpret import TokenClassificationExplainer

import time

model = None
tokenizer = None
text = None
global_html_content = ""
class_names = None


def saveToCsv(expl):
    data = []

    for target in expl.targets:
        target_id = target.target

        for feature_weight in target.feature_weights.pos:
            if feature_weight.feature == "<BIAS>":
                continue
            data.append({
                "Target": target_id,
                "Feature": feature_weight.feature,
                "Weight": feature_weight.weight,
                "Type": "Positive"
            })

        for feature_weight in target.feature_weights.neg:
            if feature_weight.feature == "<BIAS>":
                continue
            data.append({
                "Target": target_id,
                "Feature": feature_weight.feature,
                "Weight": feature_weight.weight,
                "Type": "Negative"
            })

    df = pd.DataFrame(data)

    df.to_csv('explanation.csv', index=False, encoding='utf-8')

def saveVisual(vis):
    global global_html_content
    global_html_content = vis.data
    save_html = vis.data
    with open("/content/drive/MyDrive/PhD/XAI/konn/templates/visualization.html", "w") as file:
        file.write(save_html)


class NERPredictor:
    def __init__(self):
        self.tokenizer = tokenizer
        self.model = model
        self.model.eval()

    def _predict_proba(self, texts: List[str], word_index: int, original_tokens: List[str], masked: bool):
        """
        Predicts probabilities for the given texts, aligning masked tokens if necessary.

        Parameters:
            texts (List[str]): List of text samples (masked text if masked=True, original otherwise).
            word_index (int): Index of the token being explained.
            original_tokens (List[str]): Tokenized original text (used for alignment if masked=True).
            masked (bool): Whether the text is masked (requires alignment) or not.

        Returns:
            np.array: Probabilities for the predicted token classes.
        """
        all_scores = []
        print("Len of texts when calling proba: ", len(texts))
        len_txts = []
        for txt in texts:
            len_txts.append(len(txt))

        print("Len of original text: ", len(tokenizer.convert_tokens_to_string(original_tokens)))
        print("Len of original tokens : ", len(original_tokens))
        print("Len of each text: ",len_txts)

        for txt in texts:
            batch = txt

            if masked:
                # Tokenize the masked text and align it with the original tokens
                masked_tokens = self.tokenizer.tokenize(batch)  # Tokenize the masked text
                aligned_tokens = align_masked_tokens(original_tokens, masked_tokens)  # Align them

                # Manually add special tokens
                aligned_tokens = [self.tokenizer.cls_token] + aligned_tokens + [self.tokenizer.sep_token]

                token_ids = self.tokenizer.convert_tokens_to_ids(aligned_tokens)  # Convert to IDs

                # Ensure same length by padding or truncating
                max_length = self.model.config.max_position_embeddings - 2  # Adjusted for CLS and SEP
                if len(token_ids) < max_length:
                    token_ids += [self.tokenizer.pad_token_id] * (max_length - len(token_ids))  # Pad to max length
                else:
                    token_ids = token_ids[:max_length]  # Truncate if needed

                print("Len of aligned_tokens:", len(aligned_tokens))
            else:
                # Standard tokenization without alignment
                token_ids = self.tokenizer(batch, return_tensors='pt',
                                          padding=True, truncation=True,
                                          max_length=self.model.config.max_position_embeddings-2)["input_ids"].squeeze().tolist()

            print("Final token length:", len(token_ids))
            # Convert token IDs to tensor
            encoded_input = {
                'input_ids': torch.tensor([token_ids], dtype=torch.long),
                'attention_mask': torch.tensor([[1] * len(token_ids)], dtype=torch.long)  # Full attention mask
            }

            print("from proba")
            print(encoded_input)
            print("Printing token that I'm explaining")

            """
            # Step 1: Extract the token ID at the given index
            if word_index < len(token_ids):  # Ensure the index is valid

                token_id_tensor = encoded_input['input_ids'][0, word_index]
                print(f"Token ID at index {word_index}: {token_id_tensor}")

                # Step 2: Convert token ID to token string
                token_id = token_id_tensor.item()
                token_str = self.tokenizer.convert_ids_to_tokens(token_id)
                print(f"Token at index {word_index}: {token_str}")

                if original_tokens[word_index-1] != token_str:
                  print("Tokens at word_index doesnt match")
                  print(f"Token at index {word_index} is {token_str} while original is {original_tokens[word_index-1]}")
                  print("Printing encoded masked input:")
                  token_ids = encoded_input["input_ids"][0].tolist()  # Convert tensor to list of token IDs
                  tokens = self.tokenizer.convert_ids_to_tokens(token_ids)
                  print(tokens[word_index:])
                  print("Printing original tokens:")
                  print(original_tokens)
                  raise ValueError("Tokens at word_index doesnt match")

            else:
                print(f"Warning: word_index {word_index} out of bounds for tokenized text.")
            """
            print("done proba")

            # Move input tensors to the correct device
            device = next(self.model.parameters()).device
            encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

            with torch.no_grad():
                outputs = self.model(**encoded_input)

            batch_size, sequence_length, num_labels = outputs.logits.shape
            if word_index >= sequence_length:
                probabilities = np.zeros((batch_size, num_labels))
            else:
                logits = outputs.logits[:, word_index, :]
                probabilities = torch.nn.functional.softmax(logits, dim=-1).detach().cpu().numpy()

            all_scores.extend(probabilities)

        # Adjust output shape if necessary
        expected_length = len(texts)
        actual_length = len(all_scores)
        if actual_length < expected_length:
            padding = np.zeros((expected_length - actual_length, probabilities.shape[1]))
            all_scores = np.array(all_scores)
            all_scores = np.vstack([all_scores, padding])
        else:
            all_scores = np.array(all_scores)[:expected_length]

        return all_scores


    def get_predict_function(self, word_index: int, original_tokens: List[str], masked: bool):
        def predict_func(texts: List[str]):
            return self._predict_proba(texts, word_index, original_tokens, masked)
        return predict_func

def Eli5Explainer_ent(text, predict_func, class_names):
    start_time = time.time()

    custom_pattern = r'(?u:(?:<[^>]+>)|\b\w+\b)'

    sampler = MaskingTextSampler(
    token_pattern=custom_pattern,  # default pattern
    bow=True,                    # replace one occurrence at a time
    replacement='UN',               # remove tokens
    min_replace=1,                # at least one token
    max_replace=0.5,              # potentially all tokens
    random_state=42               # for reproducibility
    )

    samples, similarity = sampler.sample_near(text, n_samples=50)

    tok_size = len(tokenizer.tokenize(text))
    samples_t = tok_size*3

    if samples_t > 5000:
      samples_t = 5000

    print("This is not eror yet")

    te = TextExplainer(
        sampler=sampler,
        position_dependent=True,
        random_state=42,
        n_samples=samples_t,
        token_pattern=custom_pattern
    )

    print("Token Numbers:", samples_t)

    te.fit(text, predict_func)

    te.class_names = class_names
    print("Class names:", class_names)
    expl = None
    vis = None
    if class_names is None:
        expl = te.explain_prediction(
            top_targets=5,
        )

        vis = te.show_prediction(show_feature_values=True)
    else:
        te.class_names = class_names
        expl = te.explain_prediction(
            top_targets=5,
            target_names=class_names,
        )

        vis = te.show_prediction(show_feature_values=True, target_names=class_names)

    end_time = time.time()
    duration = end_time - start_time

    print(f"Lime time: {duration} seconds")
    return expl, vis


def html_tags(text):
    replacements = {
        '<bold>': '&lt;bold&gt;',
        '</bold>': '&lt;/bold&gt;'
    }
    for original, escape in replacements.items():
        text = text.replace(original, escape)
    return text

def process_text(text):
    nlp = pipeline("ner", model=model, tokenizer=tokenizer)
    processed_text = ""
    last_index = 0
    word_index = 0

    id2label = nlp.model.config.id2label
    label2id = {v: k for k, v in id2label.items()}

    labe = list(nlp.model.config.id2label.values())
    first_label = labe[0]

    def split_text_into_chunks(text, max_length):
        chunks = []
        while len(text) > max_length:
            split_index = text.rfind(' ', 0, max_length)
            if split_index == -1:
                split_index = max_length
            chunks.append(text[:split_index])
            text = text[split_index:].strip()
        chunks.append(text)
        return chunks

    def process_text_chunks(text, max_length):
        chunks = split_text_into_chunks(text, max_length)
        processed_text = ""
        last_index = 0
        word_index = 0
        for chunk in chunks:

            entities = nlp(chunk)
            for entity in entities:
                start, end = entity['start'], entity['end']
                label_name = entity['entity']
                label_id = label2id.get(label_name, '0')
                entity_text = chunk[start:end]

                processed_text += chunk[last_index:start]

                if label_name != first_label:
                    processed_text += f'<span class="{label_id}" onclick="explainEntity(\'{entity_text}\', {word_index})">{entity_text}</span>'
                else:
                    processed_text += entity_text

                word_index += 1
                last_index = end

            processed_text += chunk[last_index:]
            last_index = 0

        return processed_text

    processed_text = process_text_chunks(text, 500)
    processed_text = processed_text.replace('\\n', '<br>')
    processed_text = html_tags(processed_text)

    return processed_text



def hugging_face_model_setup(model_name):
    global model, tokenizer

    tokenizer = AutoTokenizer.from_pretrained(model_name,add_prefix_space = False)
    model = AutoModelForTokenClassification.from_pretrained(model_name)

def google_model_setup(url):
    global model, tokenizer

    def extract_file_id(url):
        start = url.find('/d/') + 3
        end = url.find('/view')
        file_id = url[start:end]
        print(file_id)
        return file_id

    file_id = extract_file_id(url)
    direct_url = f'https://drive.google.com/uc?id={file_id}'

    output = 'downloaded_model.zip'
    gdown.download(direct_url, output, quiet=False)

    extract_path = './model_files'
    with zipfile.ZipFile(output, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    tokenizer = AutoTokenizer.from_pretrained("/content/model_files/tokenizer")
    model = AutoModelForTokenClassification.from_pretrained("/content/model_files/ner_model",  local_files_only=True)



port_nn = 5000
app = Flask(__name__, template_folder='/content/drive/MyDrive/PhD/XAI/konn/templates', static_folder='/content/drive/MyDrive/PhD/XAI/konn/static')
ngrok.set_auth_token("2fYaVRbruHQaiMr0joHNTLOFGjJ_4YSpxeDXJvn4kYbFtfTZN")
public_url = ngrok.connect(port_nn).public_url
app.secret_key = 'secret_key'

print (f"to acces {public_url}")

@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')

@app.route('/reset_text', methods=['GET', 'POST'])
def reset_text():
    return render_template('load_text.html', is_text_loaded=False)

@app.route('/model_loaded', methods=['POST'])
def load_model():
    model_name = request.form['model_name']

    try:
        if model_name.startswith('https://drive.google.com'):
            google_model_setup(model_name)
        else:
            hugging_face_model_setup(model_name)
        model.eval()

        return render_template('class_names.html')
    except Exception as e:
        print(str(e))

        return render_template('index.html')



@app.route('/class_names', methods=['GET', 'POST'])
def class_names():
    if request.method == 'POST':
        global class_names
        action = request.form.get('action', 'Submit')

        if action == 'Skip':
            class_names = None
            class_names = list(model.config.id2label.values())
            print("Class names with skip:", class_names)
        else:
            class_names_input = request.form.get('text', '')
            class_names = [name.strip() for name in class_names_input.split(',')] if class_names_input else None
            print("Class names:", class_names)

        return render_template('load_text.html')
    else:
        return render_template('class_names.html')


@app.route('/load_text', methods=['GET', 'POST'])
def load_text():
    global text
    if request.method == 'POST':
        file = request.files.get('file', None)
        if file and file.filename != '':
            if file.filename.endswith('.txt'):
                text = file.read().decode('utf-8')
                flash('File successfully loaded as text')
            elif file.filename.endswith('.pdf'):
                try:
                    with fitz.open(stream=file.read(), filetype='pdf') as pdf_doc:
                        pdf_text = []
                        for page in pdf_doc:
                            pdf_text.append(page.get_text())
                        text = '\n'.join(pdf_text)
                        flash('File successfully loaded as PDF')
                except Exception as e:
                    flash(f'Error reading PDF: {e}')
            else:
                flash('Only .txt and .pdf files are accepted')
        else:
            text = request.form.get('text', '').strip()
            text = text.replace('\r\n', '\n')
            print("Len of tokens : ", len(tokenizer.tokenize(text)))
            if text:
                flash('Text loaded from input')
            else:
                flash('No text or file provided')
        return redirect(url_for('load_text'))

    is_text_loaded = text is not None
    return render_template('load_text.html', is_text_loaded=is_text_loaded)


PREDEFINED_COLORS = [
    '#fd7f6f',
    '#7eb0d5',
    '#b2e061',
    '#bd7ebe',
    '#ffb55a',
    '#ffee65',
    '#beb9db',
    '#fdcce5',
    '#8bd3c7',
    '#a1dab4',
    '#abcd1f',
    '#1feabc',
    '#fe1b2c',
    '#1b2cfe',
    '#2cfe1b'
]

@app.route('/view_text')
def view_text():
    global text, class_names

    loaded_text = process_text(text) if 'text' in globals() else "No text loaded"

    if 'class_names' in globals() and class_names and len(class_names) > 1:
        filtered_class_names = class_names[1:]
        color_map = {i: PREDEFINED_COLORS[i % len(PREDEFINED_COLORS)] for i in range(len(filtered_class_names))}
        return render_template('view_text.html', loaded_text=loaded_text, class_names=enumerate(filtered_class_names, 1), color_map=color_map)
    else:
        return render_template('view_text.html', loaded_text=loaded_text, class_names=None, color_map=None)


@app.route('/transformer_text')
def transformer_text():
    global text, class_names

    start_time = time.time()

    before_names = model.config.id2label
    model.config.id2label = {i: str(class_names[i]) for i in range(len(class_names))}
    ner_explainer = TokenClassificationExplainer(
        model = model,
        tokenizer= tokenizer,
    )

    word_attributions = ner_explainer(text, ignored_labels='0')
    loaded_text=ner_explainer.visualize("transformer_ner.html")
    model.config.id2label = before_names

    end_time = time.time()
    duration = end_time - start_time

    print(f"Tran-Inter time: {duration} seconds")

    return render_template('transformer_text.html', loaded_text=loaded_text)


@app.route('/explain_entity', methods=['POST'])
def explain_ent():
    data = request.get_json()
    entity_text = data['text']
    word_index = data['index']
    print("Here I am")
    print(entity_text)
    word_index += 1
    print(word_index)

    global text
    ppp = tokenizer.tokenize(text)
    print("Token z tokenizovaneho textu ktory chcem vysvetliť: ",ppp[word_index])
    explainer_generator = NERPredictor()

    predict_func = explainer_generator.get_predict_function(word_index=word_index, original_tokens = ppp, masked = False)

    for i in range (0,len(ppp)):
        print(str(i) + ":" + ppp[i])


    print("Here I am now")
    print(predict_func([text]))
    print(tokenizer.tokenize(text)[word_index])
    print("Finish")


    custom_pattern = r'(?u:(?:<[^>]+>)|\b\w+\b)'

    """
    custom_pattern = r'(?u:\b\w+\b)'

    custom_pattern = r'(?u:(<[^>]+>|\b\w+\b))'

    toto ide ale nie je to spravne tokenizovane
    custom_pattern = r'(?u:\s*\b\w+\b)'
    """
    sampler = MaskingTextSampler(
    token_pattern=custom_pattern,  # default pattern
    bow=True,                    # replace one occurrence at a time
    replacement='UN',               # remove tokens
    min_replace=1,                # at least one token
    max_replace=0.3,              # potentially all tokens
    random_state=42               # for reproducibility
    )

    print(text)


    # Check what pattern is stored:
    print("Token pattern in sampler:", sampler.token_pattern)

    tokenized = TokenizedText(text, token_pattern=sampler.token_pattern)

    print("Tokens:", tokenized.tokens)


    samples, similarity = sampler.sample_near(text, n_samples=1)
    print("Printujem sampliky")
    print("Samples:")
    for idx,sample in enumerate(samples):
      print("This is sample number: ",idx)
      print(sample)

    tokenized_text = tokenizer.tokenize(text)
    tokenized_sample = tokenizer.tokenize(samples[0])

    print("Tokenized text:", tokenized_text)
    print("Tokenized sample:", tokenized_sample)

    aligned_tokens = align_masked_tokens(tokenized_text, tokenized_sample)
    print("Aligned tokens:", aligned_tokens)

    predict_func = explainer_generator.get_predict_function(word_index=word_index, original_tokens = ppp, masked = True)

    global class_names
    expl, vis = Eli5Explainer_ent(text, predict_func,class_names)
    saveToCsv(expl)
    saveVisual(vis)


    return jsonify(success=True)


@app.route('/download_file')
def download_file():
    file_path = '/content/explanation.csv'
    return send_file(file_path, as_attachment=True)

@app.route('/visualization')
def visualization():
    global global_html_content
    if global_html_content:
        return render_template('base_template.html', content=global_html_content)
    else:
        return "No visualization available yet", 404

if __name__ == '__main__':
    app.run(port = port_nn)

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


to acces https://a754-34-19-111-43.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 08:24:04] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 08:24:05] "GET /static/styles/main.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 08:24:05] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 08:24:13] "POST /model_loaded HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 08:24:13] "GET /static/styles/main.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 08:24:20] "POST /class_names HTTP/1.1" 200 -


Class names with skip: ['LABEL_0', 'LABEL_1', 'LABEL_2', 'LABEL_3', 'LABEL_4', 'LABEL_5', 'LABEL_6', 'LABEL_7', 'LABEL_8', 'LABEL_9', 'LABEL_10', 'LABEL_11', 'LABEL_12', 'LABEL_13', 'LABEL_14', 'LABEL_15', 'LABEL_16', 'LABEL_17', 'LABEL_18', 'LABEL_19', 'LABEL_20']


INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 08:24:20] "GET /static/styles/main.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 08:24:27] "POST /load_text HTTP/1.1" 302 -


...

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_stochastic_gradient.py:165: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Class names: ['LABEL_0', 'LABEL_1', 'LABEL_2', 'LABEL_3', 'LABEL_4', 'LABEL_5', 'LABEL_6', 'LABEL_7', 'LABEL_8', 'LABEL_9', 'LABEL_10', 'LABEL_11', 'LABEL_12', 'LABEL_13', 'LABEL_14', 'LABEL_15', 'LABEL_16', 'LABEL_17', 'LABEL_18', 'LABEL_19', 'LABEL_20']


INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 08:25:37] "POST /explain_entity HTTP/1.1" 200 -


Lime time: 64.95598125457764 seconds


INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 08:25:38] "GET /visualization HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 08:25:38] "GET /static/styles/main.css HTTP/1.1" 304 -
Device set to use cuda:0
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 08:29:48] "GET /view_text HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2025 08:29:48] "GET /static/styles/main.css HTTP/1.1" 304 -
